In [1]:
import numpy as np
import pandas as pd

In [4]:
#Import Feature Files
STUDY = 72
f_neg = pd.read_csv('study_data/reprocessed_data/IPO_aligned_MTBLS72_neg.csv', index_col=0)
f_pos = pd.read_csv('study_data/reprocessed_data/IPO_aligned_MTBLS72_pos.csv', index_col=0)

In [6]:
#import sample file
samples = pd.read_table('study_data/MTBLS72_data/s_Plasma_AD_Lipidomics.txt')

In [7]:
#map sample to disease
SAMPLE_COLUMN = 'Sample Name'
DISEASE_COLUMN = 'Factor Value[Cognitive Status]'
name_to_disease = samples[[SAMPLE_COLUMN, DISEASE_COLUMN]]
name_to_disease = name_to_disease.set_index(SAMPLE_COLUMN)

In [8]:
f_neg['drt'] = f_neg['rtmax'] - f_neg['rtmin']
f_pos['drt'] = f_pos['rtmax'] - f_pos['rtmin']
f_neg = f_neg.T
f_pos = f_pos.T
f_neg = f_neg.reset_index()
f_pos = f_pos.reset_index()

In [14]:
#Add Disease column, and drop unnecessary columns
sample_list = name_to_disease.index.tolist()
f_list = [f_neg, f_pos]
for i,features in enumerate(f_list):
    for index, row in features.iterrows():
        s = row['index']
        if '_' in s:
            #Split sample name on file by file basis
            s = s[1:].split('.mzML')[0]
            features.loc[index,'Disease']= name_to_disease.loc[s,DISEASE_COLUMN]
            features.loc[index,'new_index'] = s
        elif 'drt' in s or 'npeaks' in s or 'pcgroup' in s:
            features.loc[index, 'new_index'] = s
            features.loc[index, 'Disease'] = 'NA'
    features.dropna(inplace=True, subset=['Disease'])
    features.set_index('new_index', inplace=True)
    features.drop('index', axis = 1, inplace=True)

In [21]:
features = pd.concat([f_neg, f_pos], axis = 1)
disease = features['Disease'].iloc[:,0]
features.drop(inplace=True, labels=['Disease'], axis=1)
features = features.T.reset_index(drop=True).T
features = pd.concat([features, disease], axis = 1)

In [23]:
csv_string = 'study_'+str(STUDY)+'_processed.csv'
features.to_csv(csv_string)

In [16]:
f_neg.to_csv('f_72_neg.csv')
f_pos.to_csv('f_72_pos.csv')